In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import time
from sklearn import svm
from sklearn.metrics import classification_report

import pandas as pd

In [21]:
trainData = pd.read_csv("csvFiles/combined_csv.csv")
testData = pd.read_csv("csvFiles/April15stripped.csv")

#drop neutral rows
#indexNames = trainData[ trainData['label'] == 'neutral' ].index
#trainData.drop(indexNames , inplace=True)

In [22]:
trainData.sort_values(by=['label'])
testData.sort_values(by=['label'])
pos=trainData[trainData['label']=="pos"]
neg=trainData[trainData['label']=="neg"]
neutral=trainData[trainData['label']=="neutral"]

In [23]:
print(pos.count())
print(neg.count())
print(neutral.count())

comment    52
label      52
dtype: int64
comment    59
label      59
dtype: int64
comment    504
label      504
dtype: int64


In [31]:
sample_size= min(pos.shape[0],neutral.shape[0],neg.shape[0])
print(sample_size)

52


In [40]:
pos=pos.sample(sample_size)
neg=neg.sample(sample_size)
neutral=neutral.sample(sample_size)

In [41]:
combine=[pos,neg,neutral]
result= pd.concat(combine)
result

,comment,label
316,Gap up to $300 spy right?,pos
236,why do i have a bad feeling the past 2 days we...,pos
575,AMZN will hit $3k soon.\n\n\- Massive conversi...,pos
598,I'm gonna quit my job so unemployment goes up....,pos
408,Dick n ball shaped recovery confirmed,pos
...,...,...
201,They just stopped bringing Fauci to this?,neutral
246,This “R Fuk” nonsense has to stop,neutral
17,Can I interest any of you fine gentlemen in so...,neutral
524,[this idiot bought $1.2m in amzn shares when h...,neutral


In [42]:
vectorizer = TfidfVectorizer(min_df = 5,
                             max_df = 0.8,
                             sublinear_tf = True,
                             use_idf = True)



train_vectors = vectorizer.fit_transform(result['comment'])
test_vectors = vectorizer.transform(testData['comment'])

In [44]:
classifier_linear = svm.SVC(kernel='linear')
t0 = time.time()
classifier_linear.fit(train_vectors, result['label'])
t1 = time.time()
prediction_linear = classifier_linear.predict(test_vectors)
t2 = time.time()
time_linear_train = t1-t0
time_linear_predict = t2-t1

In [45]:
# results
print("Results for SVC(kernel=linear)")
print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))
report = classification_report(testData['label'], prediction_linear, output_dict=True)
print('positive: ', report['pos'])
print('negative: ', report['neg'])
print('neutral: ', report['neutral'])

Results for SVC(kernel=linear)
Training time: 0.003982s; Prediction time: 0.000998s
positive:  {'precision': 0.4375, 'recall': 0.3333333333333333, 'f1-score': 0.37837837837837834, 'support': 21}
negative:  {'precision': 0.21428571428571427, 'recall': 0.5454545454545454, 'f1-score': 0.30769230769230765, 'support': 11}
neutral:  {'precision': 0.6333333333333333, 'recall': 0.4523809523809524, 'f1-score': 0.5277777777777778, 'support': 42}
